# Association rule mining

In this notebook, we will look at the task of learning association rules, that is, learning rule of the type "customers who bought A and B, also bought C", for instance. 

It is a classical data mining task to find out what items are frequently bought together is also known as *Market Basket Analysis*. Association rule mining is one approach to this. There are other methods for solving this and similar tasks such as recommender systems or collaborative filtering

Applications of association rule mining is product recommendation such as on Amazon or Netflix, for instance. (What happens at Amazon and Netflix is of course more advanced than just simple association rules mining we will see here.) Other applications could be placing product next to each other in a physical store (for instance the classical example of diapers and beers being bought together) or devising offers and advertisement. 

## Defining Association rules mining

For association rule mining, data is given as a set of "baskets"/"transactions" each containing a set of items. Association rules mining then tries to generate rules of the form *"if a customer has oatmeal and sugar in her basket, it is likely that she also has milk in her basket"*

We will use the `arules` package for association rules learning in R, so let us install and load it:

In [1]:
install.packages("arules")
library(arules)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Loading required package: Matrix

Attaching package: ‘arules’

The following objects are masked from ‘package:base’:

    abbreviate, write



As an example, we will use the dataset "Groceries" from the `arules` package.

In [2]:
data("Groceries")
str(Groceries)

Formal class 'transactions' [package "arules"] with 3 slots
  ..@ data       :Formal class 'ngCMatrix' [package "Matrix"] with 5 slots
  .. .. ..@ i       : int [1:43367] 13 60 69 78 14 29 98 24 15 29 ...
  .. .. ..@ p       : int [1:9836] 0 4 7 8 12 16 21 22 27 28 ...
  .. .. ..@ Dim     : int [1:2] 169 9835
  .. .. ..@ Dimnames:List of 2
  .. .. .. ..$ : NULL
  .. .. .. ..$ : NULL
  .. .. ..@ factors : list()
  ..@ itemInfo   :'data.frame':	169 obs. of  3 variables:
  .. ..$ labels: chr [1:169] "frankfurter" "sausage" "liver loaf" "ham" ...
  .. ..$ level2: Factor w/ 55 levels "baby food","bags",..: 44 44 44 44 44 44 44 42 42 41 ...
  .. ..$ level1: Factor w/ 10 levels "canned food",..: 6 6 6 6 6 6 6 6 6 6 ...
  ..@ itemsetInfo:'data.frame':	0 obs. of  0 variables


Note that the Groceries dataset is not a data frame as we are used to. It a special kind of object/class called "transactions", which is a special class of the `arules` package that makes the data easily available for association rule mining. So to get a better grip of the data, instead of the `str` function we can use the `inspect` function or the classic `summary` function.

In [3]:
inspect(Groceries[1:6, ])

    items                     
[1] {citrus fruit,            
     semi-finished bread,     
     margarine,               
     ready soups}             
[2] {tropical fruit,          
     yogurt,                  
     coffee}                  
[3] {whole milk}              
[4] {pip fruit,               
     yogurt,                  
     cream cheese ,           
     meat spreads}            
[5] {other vegetables,        
     whole milk,              
     condensed milk,          
     long life bakery product}
[6] {whole milk,              
     butter,                  
     yogurt,                  
     rice,                    
     abrasive cleaner}        


Note how the transaction data simply consists of customer transactions of "baskets" of items bought together. (Information about the quantity of each item is abstracted away in this representation as it is not needed for the task.)

In [4]:
summary(Groceries)

transactions as itemMatrix in sparse format with
 9835 rows (elements/itemsets/transactions) and
 169 columns (items) and a density of 0.02609146 

most frequent items:
      whole milk other vegetables       rolls/buns             soda 
            2513             1903             1809             1715 
          yogurt          (Other) 
            1372            34055 

element (itemset/transaction) length distribution:
sizes
   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
2159 1643 1299 1005  855  645  545  438  350  246  182  117   78   77   55   46 
  17   18   19   20   21   22   23   24   26   27   28   29   32 
  29   14   14    9   11    4    6    1    1    1    1    3    1 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   2.000   3.000   4.409   6.000  32.000 

includes extended item information - examples:
       labels  level2           level1
1 frankfurter sausage meat and sausage
2     sausage sausage meat and sausage
3  live

The summary give us some summary information about the data, not surprisingly. It tells us the number of transactions/baskets (9835), and the total number of items (169). I tells us the most frequent items, i.e. how many transactions to most frequent items occurs in. I also give us a distribution of the sizes of each transaction/basket ranging from 1 to 32 number of items with a mean 4.4 items per basket. (We can also see that the transactions are represented as a itemMatrix object in sparse format.)

We can also investigate the data further by asking how frequent do the different items occur, for instance:

In [5]:
options(repr.plot.width=8, repr.plot.height=6)
itemFrequency(Groceries)[1:4]
itemFrequencyPlot(Groceries, support = 0.07)

frankfurter     sausage  liver loaf         ham 
0.058973055 0.093950178 0.005083884 0.026029487

This plot show us the frequency of all items that have a frequency above 0.07 (- the `support = 0.07` argument in the call to `itemFrequencyPlot`). We can see that the most frequently bought item is whole milk with a frequency just above 0.25.

## Association rules mining terminology

Some terminology is important here, so let us introduce it:

* **Association rule**. An *association rule* is of the form *"If LHS, then RHS"*, where LHS and RHS are both sets of items.
    + For example: The rule "if {milk, chocolate} then {chili}" say that if milk and chocolate occurs together in a basket, then the basket is likely to contain chili as well.
* **Support**: The *support* of a set of items is the proportion of all baskets where the particular combination of items occurred.
    + For example: if `support({milk, chocolate}) = 0.01`, it means that milk and chocolate occurred together in `1%` of the baskets.
    + The *support* of a rule "if LHS, then RHS" is the support of the item set `{LHS, RHS}`, i.e. `support("if LHS, then RHS") = support({LHS, RHS})`
    + High support tell us that the combination is frequent.
    + To low support of a rule tell us that that there can be high uncertainty wrt. the rule, but also that we will not have many chances of applying the rule, so it is not really an interesting rule
    + If both the `LHS` and the `RHS` of a rule have high support, the rule might be an expression of this instead of an actual rule. If a lot of people often buy milk and a lot of people often buy bread, milk and bread will appear together often without it being a real "association"
* **Confidence**: The confidence of a rule is the proportion of times the `RHS` of a rule occurs when the `LHS` of the rule occurs.
    + For example: `confidence("if {milk, chocolate} then {chili}") = 0.5` means that `50%` of the times milk and chocolate occurred together, chili also occurred.
    + In other words, it is the number of baskets containing both `LHS` and `RHS` divided with the number of baskets containing `LHS`
    + Alternatively, in math: `confidence("if LHS, then RHS") = support({LHS, RHS})/support(LHS)`
    + High confidence says something about how applicable the rule is -- how confident we can be in it

* **Lift** is another measure of the "quality" of an association rule
    + Greater lift values indicate stronger associations.
    + The definition:
`lift("If LHS, then RHS") = support({LHS, RHS}) / (support(LHS) * support(RHS))`
    + Alternatively: `lift("if LHS, then RHS") = confidence("if LHS, then RHS") / support(RHS)`
    + Intuitively, the lift is the "lift" in the likelihood of `RHS` if we know `LHS` compared to the likelihood of `RHS` in general case

## Finding association rules

One of the most common algorithms for finding association rules is the *aprior algorithm*. In the "arules" package this algorithm is implemented in the `apriori` function. To apply the `apriori` function in R, we need to set a minimum for support and confidence. These limits will depend on the data and the domain.
* We want a support that is low, but still make the potential rule useful (`LHS` will occur often enough for us to recommend the `RHS`)
* We want confidence high such that there is a fair amount of certainty in the rule.
* However, in the end, the rules with the highest lift are likely the most interesting ones

The `apriori` function can now be called in the following way:

In [6]:
apriori(Groceries, parameter = list(support = 0.01, confidence = 0.5))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.5    0.1    1 none FALSE            TRUE       5    0.01      1
 maxlen target   ext
     10  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 98 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.01s].
sorting and recoding items ... [88 item(s)] done [0.00s].
creating transaction tree ... done [0.01s].
checking subsets of size 1 2 3 4 done [0.00s].
writing ... [15 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


set of 15 rules 

We can also try other parameter values:

In [7]:
apriori(Groceries, parameter = list(support = 0.005, confidence = 0.5))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.5    0.1    1 none FALSE            TRUE       5   0.005      1
 maxlen target   ext
     10  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 49 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.01s].
sorting and recoding items ... [120 item(s)] done [0.00s].
creating transaction tree ... done [0.01s].
checking subsets of size 1 2 3 4 done [0.01s].
writing ... [120 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


set of 120 rules 

In [8]:
apriori(Groceries, parameter = list(support = 0.005, confidence = 0.6))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.6    0.1    1 none FALSE            TRUE       5   0.005      1
 maxlen target   ext
     10  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 49 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.01s].
sorting and recoding items ... [120 item(s)] done [0.00s].
creating transaction tree ... done [0.01s].
checking subsets of size 1 2 3 4 done [0.01s].
writing ... [22 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


set of 22 rules 

Let us stick to a support of 0.01 and a confidence of 0.5 and investigate the rules it returns:

In [9]:
rules <- apriori(Groceries, parameter = list(support = 0.01, confidence = 0.5))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.5    0.1    1 none FALSE            TRUE       5    0.01      1
 maxlen target   ext
     10  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 98 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.01s].
sorting and recoding items ... [88 item(s)] done [0.00s].
creating transaction tree ... done [0.01s].
checking subsets of size 1 2 3 4 done [0.00s].
writing ... [15 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


In [10]:
rules
inspect(rules[1:4])

set of 15 rules 

    lhs                                 rhs          support    confidence
[1] {curd,yogurt}                    => {whole milk} 0.01006609 0.5823529 
[2] {other vegetables,butter}        => {whole milk} 0.01148958 0.5736041 
[3] {other vegetables,domestic eggs} => {whole milk} 0.01230300 0.5525114 
[4] {yogurt,whipped/sour cream}      => {whole milk} 0.01087951 0.5245098 
    lift     count
[1] 2.279125  99  
[2] 2.244885 113  
[3] 2.162336 121  
[4] 2.052747 107  


In [11]:
summary(rules)

set of 15 rules

rule length distribution (lhs + rhs):sizes
 3 
15 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      3       3       3       3       3       3 

summary of quality measures:
    support          confidence          lift           count      
 Min.   :0.01007   Min.   :0.5000   Min.   :1.984   Min.   : 99.0  
 1st Qu.:0.01174   1st Qu.:0.5151   1st Qu.:2.036   1st Qu.:115.5  
 Median :0.01230   Median :0.5245   Median :2.203   Median :121.0  
 Mean   :0.01316   Mean   :0.5411   Mean   :2.299   Mean   :129.4  
 3rd Qu.:0.01403   3rd Qu.:0.5718   3rd Qu.:2.432   3rd Qu.:138.0  
 Max.   :0.02227   Max.   :0.5862   Max.   :3.030   Max.   :219.0  

mining info:
      data ntransactions support confidence
 Groceries          9835    0.01        0.5

From this we can see that we got 1 rule of length 2, 98 rules of length 3 and 21 rules of length 4. We also get some descriptive statistics on the support, confidence and lift of these total of 120 rules.

We can now sort the rules by their lift to find the most valuable ones. (We turn it into a data frame for easier reading in Jupyter Notebooks.)

In [12]:
as.data.frame(inspect(sort(rules, by = "lift")))

     lhs                                      rhs                support   
[1]  {citrus fruit,root vegetables}        => {other vegetables} 0.01037112
[2]  {tropical fruit,root vegetables}      => {other vegetables} 0.01230300
[3]  {root vegetables,rolls/buns}          => {other vegetables} 0.01220132
[4]  {root vegetables,yogurt}              => {other vegetables} 0.01291307
[5]  {curd,yogurt}                         => {whole milk}       0.01006609
[6]  {other vegetables,butter}             => {whole milk}       0.01148958
[7]  {tropical fruit,root vegetables}      => {whole milk}       0.01199797
[8]  {root vegetables,yogurt}              => {whole milk}       0.01453991
[9]  {other vegetables,domestic eggs}      => {whole milk}       0.01230300
[10] {yogurt,whipped/sour cream}           => {whole milk}       0.01087951
[11] {root vegetables,rolls/buns}          => {whole milk}       0.01270971
[12] {pip fruit,other vegetables}          => {whole milk}       0.01352313
[13] {tropic

,lhs,,rhs,support,confidence,lift,count
[1],"{citrus fruit,root vegetables}",=>,{other vegetables},0.01037112,0.5862069,3.029608,102
[2],"{tropical fruit,root vegetables}",=>,{other vegetables},0.01230300,0.5845411,3.020999,121
[3],"{root vegetables,rolls/buns}",=>,{other vegetables},0.01220132,0.5020921,2.594890,120
[4],"{root vegetables,yogurt}",=>,{other vegetables},0.01291307,0.5000000,2.584078,127
[5],"{curd,yogurt}",=>,{whole milk},0.01006609,0.5823529,2.279125,99
[6],"{other vegetables,butter}",=>,{whole milk},0.01148958,0.5736041,2.244885,113
[7],"{tropical fruit,root vegetables}",=>,{whole milk},0.01199797,0.5700483,2.230969,118
[8],"{root vegetables,yogurt}",=>,{whole milk},0.01453991,0.5629921,2.203354,143
[9],"{other vegetables,domestic eggs}",=>,{whole milk},0.01230300,0.5525114,2.162336,121
[10],"{yogurt,whipped/sour cream}",=>,{whole milk},0.01087951,0.5245098,2.052747,107


### Exercise

The `arules` comes with another dataset named `Epub`. Load that dataset using `data(Epub)` and do association rule mining on it. What are the three rules with the most lift?

## Example involving transaction data

Unti now we have only used data that was already in the right format for using the `arules` package. However, this is not how real data comes. So in this example we will look at som typical transaction data, load it into R and transform it into a proper format that suits the `arules` package.

We will load a typical transaction dataset found on the Tableau Community forum: https://community.tableau.com/docs/DOC-1236

In [13]:
library(readxl)
url <- "https://community.tableau.com/servlet/JiveServlet/downloadBody/1236-102-2-15278/Sample%20-%20Superstore.xls"
tempdestfile <- "temp.xls"
download.file(url, tempdestfile)
transactionData <- read_excel(tempdestfile)

Warning message in read_fun(path = path, sheet = sheet, limits = limits, shim = shim, :
“Coercing text to numeric in L2236 / R2236C12: '05408'”Warning message in read_fun(path = path, sheet = sheet, limits = limits, shim = shim, :
“Coercing text to numeric in L5276 / R5276C12: '05408'”Warning message in read_fun(path = path, sheet = sheet, limits = limits, shim = shim, :
“Coercing text to numeric in L8800 / R8800C12: '05408'”Warning message in read_fun(path = path, sheet = sheet, limits = limits, shim = shim, :
“Coercing text to numeric in L9148 / R9148C12: '05408'”Warning message in read_fun(path = path, sheet = sheet, limits = limits, shim = shim, :
“Coercing text to numeric in L9149 / R9149C12: '05408'”Warning message in read_fun(path = path, sheet = sheet, limits = limits, shim = shim, :
“Coercing text to numeric in L9150 / R9150C12: '05408'”Warning message in read_fun(path = path, sheet = sheet, limits = limits, shim = shim, :
“Coercing text to numeric in L9388 / R9388C12: '05408'

In [14]:
head(transactionData)

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.9400,3,0.00,219.5820
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.6200,2,0.00,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood",48.8600,7,0.00,14.1694


We are only interested in the `Order ID` and the `Product ID` column as this will tell us which products are bought together in an order. Thereby we can recreate the baskets for an association rule mining task. We do a little clearning of the data also.

In [15]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.1     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::recode() masks arules::recode()


In [16]:
transactionData <- transactionData %>% select('Order ID', 'Product ID')

In [17]:
head(transactionData)

Order ID,Product ID
CA-2016-152156,FUR-BO-10001798
CA-2016-152156,FUR-CH-10000454
CA-2016-138688,OFF-LA-10000240
US-2015-108966,FUR-TA-10000577
US-2015-108966,OFF-ST-10000760
CA-2014-115812,FUR-FU-10001487


In [18]:
names(transactionData) <- c("OrderID", "ProductID")

In [19]:
str(transactionData)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	9994 obs. of  2 variables:
 $ OrderID  : chr  "CA-2016-152156" "CA-2016-152156" "CA-2016-138688" "US-2015-108966" ...
 $ ProductID: chr  "FUR-BO-10001798" "FUR-CH-10000454" "OFF-LA-10000240" "FUR-TA-10000577" ...


To transformt the data into a proper transaction class for the `aprior` algorithm, it is easiest to write the data out to a file and load it in using the `read.transactions` function.

In [20]:
write.csv(transactionData, file = "afile.csv", row.names=FALSE)

In [21]:
transData <- read.transactions("afile.csv", format = "single", cols = c(1,2))

By inspecting the data we can see that it is now in a familiar format we can work with and use as input for the `apriori` function:

In [22]:
inspect(transData[1:4])

    items                                   transactionID 
[1] {,"TEC-PH-10002075"}                    CA-2014-100006
[2] {,"FUR-TA-10003715",,"OFF-BI-10001597"} CA-2014-100090
[3] {,"OFF-PA-10000176"}                    CA-2014-100293
[4] {,"OFF-BI-10000343"}                    CA-2014-100328


In [23]:
summary(transData)

transactions as itemMatrix in sparse format with
 5010 rows (elements/itemsets/transactions) and
 1863 columns (items) and a density of 0.001070002 

most frequent items:
,"OFF-PA-10001970" ,"TEC-AC-10003832" ,"FUR-FU-10004270" ,"FUR-CH-10001146" 
                18                 18                 16                 15 
,"FUR-CH-10002647"            (Other) 
                15               9905 

element (itemset/transaction) length distribution:
sizes
   1    2    3    4    5    6    7    8    9   10   11   12   14 
2541 1218  607  333  157   70   52   15   10    3    2    1    1 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   1.000   1.993   2.000  14.000 

includes extended item information - examples:
              labels
1 ,"FUR-BO-10000112"
2 ,"FUR-BO-10000330"
3 ,"FUR-BO-10000362"

includes extended transaction information - examples:
   transactionID
1 CA-2014-100006
2 CA-2014-100090
3 CA-2014-100293

### Exercise

1. For the `transData` data make item frequency plot
2. Do association rule mining on the `transData` using the `apriori` function